In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [3]:
# Get all files in the Details folder
details_path = "../Data/Input/Events/"
files = [f"{details_path}{filename}" for filename in os.listdir(details_path) if os.path.isfile((os.path.join(details_path, filename)))]

In [5]:
output_path = "../Data/Output/"

In [6]:
# Timezone data
tz_codes = ['CST', 'MST', 'EST', 'PST', 'UNK', 'CDT', 'EDT', 'MDT', 'GMT', 'HST', 'PDT', 'CSC',
 'AST', 'EST-5', 'MST-7', 'CST-6', 'PST-8', 'AST-4', 'HST-10', 'AKST-9']
tz_offsets = [-6, -7, -5, -8, 0, -5, -4, -6, 0, -10, -7, -6, -4, -5, -7, -6, -8, -4, -10, -9]
tz_recode = ['CST-6', 'MST-7', 'EST-5', 'PST-8', 'UNK', 'CDT-5', 'EDT-4', 'MDT-6', 'GMT-0',
             'HST-10', 'PDT-7', 'CST-6', 'AST-4', 'EST-5', 'MST-7', 'CST-6', 'PST-8', 'AST-4',
             'HST-10', 'AKST-9']
timezones_df = pd.DataFrame({
    "CZ_TIMEZONE": tz_codes,
    "CZ_TIMEZONE_RECODE": tz_recode,
    "UTC_OFFSET": tz_offsets
})

timezones_df

,CZ_TIMEZONE,CZ_TIMEZONE_RECODE,UTC_OFFSET
0,CST,CST-6,-6
1,MST,MST-7,-7
2,EST,EST-5,-5
3,PST,PST-8,-8
4,UNK,UNK,0
5,CDT,CDT-5,-5
6,EDT,EDT-4,-4
7,MDT,MDT-6,-6
8,GMT,GMT-0,0
9,HST,HST-10,-10


In [94]:
def process_file(file):
    print(f"Processing file {file}")
    # Read file. Set FIPS columns to string
    detail_data_raw_df = pd.read_csv(file, dtype={"STATE_FIPS": str,
                                                  "CZ_FIPS": str,
                                                  "TOR_OTHER_CZ_FIPS": str,
                                                  "DAMAGE_PROPERTY": str,
                                                  "DAMAGE_CROPS": str,
                                                  "BEGIN_AZIMUTH": str,
                                                  "BEGIN_LOCATION": str,
                                                  "END_AZIMUTH": str,
                                                  "END_LOCATION": str})
    
    # Filter tornadoes
    detail_data_clean_df = detail_data_raw_df[detail_data_raw_df["EVENT_TYPE"] == "Tornado"]
    
    # Renumber Puerto Rico & Virgin Islands state FIPS
    detail_data_clean_df.loc[detail_data_clean_df["STATE"] == "PUERTO RICO", "STATE_FIPS"] = "72"
    detail_data_clean_df.loc[detail_data_clean_df["STATE"] == "VIRGIN ISLANDS", "STATE_FIPS"] = "78"

    # Combine State and County FIPS
    detail_data_clean_df["FIPS"] = (detail_data_clean_df["STATE_FIPS"].str.zfill(2)) + (detail_data_clean_df["CZ_FIPS"].str.zfill(3))
    
    # Process timestamps
    pattern = r'-\d{2}'
    detail_data_clean_df["YEAR"] = detail_data_clean_df["YEAR"].astype(str)
 
    # FROM MICROSOFT COPILOT
    # Replace 2-digit year with 4-digit year for timestamp creation
    detail_data_clean_df["BEGIN_DATE_TIME"] = detail_data_clean_df.apply(
        lambda row: pd.Series(row["BEGIN_DATE_TIME"]).str.replace(pattern, f"-{row["YEAR"]}", regex=True)[0],axis=1)
    detail_data_clean_df["END_DATE_TIME"] = detail_data_clean_df.apply(
        lambda row: pd.Series(row["END_DATE_TIME"]).str.replace(pattern, f"-{row["YEAR"]}", regex=True)[0],axis=1)
    
    # Convert BEGIN_DATE_TIME and END_DATE_TIME to UTC UNIX (POSIX) timestamps
    detail_data_clean_df.loc[:,"B_DATE_TIME"] = pd.to_datetime(detail_data_clean_df.loc[:,"BEGIN_DATE_TIME"],
                                                               format="%d-%b-%Y %H:%M:%S")
    detail_data_clean_df.loc[:,"E_DATE_TIME"] = pd.to_datetime(detail_data_clean_df.loc[:,"END_DATE_TIME"],
                                                               format="%d-%b-%Y %H:%M:%S")
    
    detail_data_clean_df = detail_data_clean_df.merge(timezones_df, on="CZ_TIMEZONE", how="left")

    detail_data_clean_df["B_DATE_TIME"] = detail_data_clean_df["B_DATE_TIME"]\
                                        - pd.TimedeltaIndex(detail_data_clean_df["UTC_OFFSET"], unit="H")
    detail_data_clean_df["E_DATE_TIME"] = detail_data_clean_df["E_DATE_TIME"]\
                                        - pd.TimedeltaIndex(detail_data_clean_df["UTC_OFFSET"], unit="H")
    
    detail_data_clean_df.loc[:,"BEGIN_TIMESTAMP"] = detail_data_clean_df.loc[:,"B_DATE_TIME"].astype("int64") // 10**9
    detail_data_clean_df.loc[:,"END_TIMESTAMP"] = detail_data_clean_df.loc[:,"E_DATE_TIME"].astype("int64") // 10**9

    # Get F/EF scale codes
    detail_data_clean_df["TOR_F_SCALE"] = detail_data_clean_df["TOR_F_SCALE"].fillna("EFU")
    pattern = r'F(\w)'
    detail_data_clean_df["TOR_F_LEVEL"] = detail_data_clean_df["TOR_F_SCALE"].str.extract(pattern)

    # Accumulate Deaths & Injuries
    detail_data_clean_df["DEATHS"] = detail_data_clean_df["DEATHS_DIRECT"] + detail_data_clean_df["DEATHS_INDIRECT"]
    detail_data_clean_df["INJURIES"] = detail_data_clean_df["INJURIES_DIRECT"] + detail_data_clean_df["INJURIES_INDIRECT"]

    # Convert DAMAGE_PROPERTY to numeric
    detail_data_clean_df.loc[:,"DAMAGE_PROPERTY"] = detail_data_clean_df.loc[:,"DAMAGE_PROPERTY"].fillna("0.00K")
    detail_data_clean_df.loc[detail_data_clean_df["DAMAGE_PROPERTY"].str.match(r'\d*[.]?\d*[^KMB]\Z'), "DAMAGE_PROPERTY"] += "K"

    pattern = r'(\d*[.]?\d*)[KMB]'
    detail_data_clean_df["DMG_PRP"] = detail_data_clean_df["DAMAGE_PROPERTY"].str.extract(pattern).astype(float)
    pattern = r'\d*[.]?\d*([KMB])'
    detail_data_clean_df["DMG_PRP_MULT_STR"] = detail_data_clean_df["DAMAGE_PROPERTY"].str.extract(pattern)
    detail_data_clean_df["DMG_PRP"] = (detail_data_clean_df["DMG_PRP"] * 
                                        np.where(detail_data_clean_df["DMG_PRP_MULT_STR"] == "K", 1000, 1))
    detail_data_clean_df["DMG_PRP"] = (detail_data_clean_df["DMG_PRP"] * 
                                        np.where(detail_data_clean_df["DMG_PRP_MULT_STR"] == "M", 1000000, 1))
    detail_data_clean_df["DMG_PRP"] = (detail_data_clean_df["DMG_PRP"] * 
                                        np.where(detail_data_clean_df["DMG_PRP_MULT_STR"] == "B", 1000000000, 1))

    # Convert DAMAGE_CROPS to numeric
    detail_data_clean_df.loc[:,"DAMAGE_CROPS"] = detail_data_clean_df.loc[:,"DAMAGE_CROPS"].fillna("0.00K")
    detail_data_clean_df.loc[detail_data_clean_df["DAMAGE_CROPS"].str.match(r'\d*[.]?\d*[^KMB]\Z'), "DAMAGE_CROPS"] += "K"

    pattern = r'(\d*[.]?\d*)[KMB]'
    detail_data_clean_df["DMG_CRP"] = detail_data_clean_df["DAMAGE_CROPS"].str.extract(pattern).astype(float)
    pattern = r'\d*[.]?\d*([KMB])'
    detail_data_clean_df["DMG_CRP_MULT_STR"] = detail_data_clean_df["DAMAGE_CROPS"].str.extract(pattern)
    detail_data_clean_df["DMG_CRP"] = (detail_data_clean_df["DMG_CRP"] * 
                                        np.where(detail_data_clean_df["DMG_CRP_MULT_STR"] == "K", 1000, 1))
    detail_data_clean_df["DMG_CRP"] = (detail_data_clean_df["DMG_CRP"] * 
                                        np.where(detail_data_clean_df["DMG_CRP_MULT_STR"] == "M", 1000000, 1))
    detail_data_clean_df["DMG_CRP"] = (detail_data_clean_df["DMG_CRP"] * 
                                        np.where(detail_data_clean_df["DMG_CRP_MULT_STR"] == "B", 1000000000, 1))
    
    # Trim columns
    detail_data_clean_df = detail_data_clean_df[[
       'EVENT_ID', 'FIPS',
       'BEGIN_TIMESTAMP', 'END_TIMESTAMP',
       'DEATHS', 'INJURIES', 'DMG_PRP', 'DMG_CRP',
       'TOR_F_SCALE', 'TOR_F_LEVEL', 'TOR_LENGTH', 'TOR_WIDTH',        
       'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION',
       'END_RANGE', 'END_AZIMUTH', 'END_LOCATION',
       'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EVENT_NARRATIVE']]
    
    # Rename temporary columns back to their original names
    detail_data_clean_df = detail_data_clean_df.rename({"DMG_PRP": "DAMAGE_PROPERTY",
                                                        "DMG_CRP": "DAMAGE_CROPS" }, axis=1)

    # Fix numeric dtypes
    detail_data_clean_df["DAMAGE_PROPERTY"] = detail_data_clean_df["DAMAGE_PROPERTY"].astype("int64")
    detail_data_clean_df["DAMAGE_CROPS"] = detail_data_clean_df["DAMAGE_CROPS"].astype("int64")

    return detail_data_clean_df


In [97]:
enabled = True
write = True

if enabled:
    df_list = [process_file(file) for file in files]
    details_full_df = pd.concat(df_list)
    details_full_df = details_full_df.sort_values("BEGIN_TIMESTAMP")
    details_full_df = details_full_df.reset_index()
    details_full_df = details_full_df.drop(columns=["index"])
    print("File processing complete")

    if write:
        details_full_df.to_csv(f"{output_path}/Tornadoes_1950_2024.csv", index=False)        
        print(f"File written to {output_path}/Tornadoes_1950_2024.csv")        

Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1950_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1951_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1952_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1953_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1954_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1955_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1956_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1957_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1958_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1959_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1960_c20210803.csv
Processing file ../Data/Details/StormEvents_details-ftp_v1.0_d1961_c20210803.csv
Processing file ../Data/Deta

In [166]:
# Check that the FIPS columns matches the current FIPS data
fips_path = "../Data/Output/fips_data.csv"
fips_df = pd.read_csv(fips_path, dtype=str)
fips_list = fips_df["FIPS"].tolist()

details_full_df[~details_full_df["FIPS"].isin(fips_list)]

,EVENT_ID,FIPS,BEGIN_TIMESTAMP,END_TIMESTAMP,DEATHS,INJURIES,DAMAGE_PROPERTY,DAMAGE_CROPS,TOR_F_SCALE,TOR_F_LEVEL,...,BEGIN_AZIMUTH,BEGIN_LOCATION,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EVENT_NARRATIVE
